In [ ]:
####IMPORT MODULES
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import random
import openpyxl

####IMPORT LOCAL
from header_list import headers

In [ ]:
####READ IN

##get the excel-file as a workbook
wb = openpyxl.load_workbook("data/Depot.xlsx")

##we expect the list of names to operate on to be on the first sheet, column A
sheet = wb.worksheets[0]

#if it's the first time running, create a StockPriceUpdate-sheet
if "StockPriceUpdate" not in wb.sheetnames:
    update_sheet = wb.create_sheet("StockPriceUpdate")
    first_time = True

#when it exists, we refer to it as update_sheet
update_sheet = wb["StockPriceUpdate"]

#StockPriceUpdate will have the names in column A and the updated prices in column B. All old data will be at C onwards.
update_sheet.insert_cols(idx=1)
update_sheet.insert_cols(idx=2)

##update the header
update_sheet["A1"] = "names"

update_sheet["B1"] = f'{datetime.now().strftime("Data %d/%m/%Y %H:%M")}'




In [ ]:
####FUNCTIONS
def make_url(name):
    url = f"https://www.finanzen.net/aktien/{name}-aktie"
    return url

def get_price(url):
    try:
        response = requests.get(url, headers=random.choice(headers), timeout=10)
        response.raise_for_status()  
        
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:  
            print(f"404: Skipping {url}")
            return None  
        else:
            raise  
            
    except (requests.exceptions.RequestException, 
            requests.exceptions.Timeout,
            requests.exceptions.SSLError) as e:
        print(f"Network error ({type(e).__name__}): Skipping {url}")
        return None
        
    try:
        soup = BeautifulSoup(response.text, 'html.parser')
        span_tag = soup.find('span', class_='snapshot__value')
        number = span_tag.get_text(strip=True)
        return number
        
    except AttributeError: 
        print(f"Missing price element: Skipping {url}")
        return None



In [ ]:
##row=2 to skip header, start=2 to preserve identity of rows

for cell_value in sheet.iter_cols(min_col=1, max_col=1, min_row=2, values_only=True):
    for row_idx, stock_name in enumerate(cell_value, start=2):
        if stock_name is not None and stock_name != "NaN":
            url = make_url(stock_name)
            print(url)
            price = get_price(url)
            update_sheet[f"A{row_idx}"] = stock_name
            update_sheet[f"B{row_idx}"] = price

In [ ]:
####SAVE IT

date=datetime.now().strftime("Data %d_%m_%Y-%H:%M")
wb.save(f'updated_depot_{date}.xlsx')
